<a href="https://colab.research.google.com/github/akilaIduwara/akilaiduwara/blob/main/Akila_Induwara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Underground Water Detect Algorithms

In [ ]:
#Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Enable Mixed Precision (if using GPU)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
# Step 2: Load and Preprocess Data Efficiently
try:
    df = pd.read_csv('/content/JWD_L2B_DD_ERTmodel.csv', skiprows=1, header=0)
    df.columns = df.columns.str.strip()
    print("Columns in the CSV file:", df.columns.tolist())

    if 'Log10Res' not in df.columns:
        raise KeyError("Column 'Log10Res' not found in the CSV file. Please check the column names.")
    # Filter invalid values
    df = df[df['Log10Res'] != -9999]
    df['Resistivity'] = 10 ** df['Log10Res']
    df['is_water'] = np.where((df['Resistivity'] >= 20) & (df['Resistivity'] <= 100), 1, 0)

    features = df[['Dist', 'X_NAD83UTMz16N', 'Y_NAD83UTMz16N', 'Elev']]
    target = df['is_water']

except Exception as e:
    print("Error during data loading or preprocessing:", e)
    exit()



In [ ]:
# Step 3: Train ANN Deep Learning Model with Large Data Handling
try:
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.2, random_state=42
    )
 # Normalize features
    mean = X_train.mean()
    std = X_train.std()
    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std

 # Convert to TensorFlow datasets for efficiency
    batch_size = 64
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values)).shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

 # Define ANN model
    model = keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(0.3),  # Prevents overfitting
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

 # Early stopping callback to prevent overtraining
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

 # Train the model
    model.fit(train_dataset, epochs=30, validation_data=test_dataset, callbacks=[early_stopping])


 # Evaluate model
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

except Exception as e:
    print("Error during model training or evaluation:", e)
    exit()



In [ ]:
# Step 4: Identify Water Resources and Display Results Efficiently
try:
    # Normalize the full dataset
    df_normalized = (features - mean) / std
    dataset_full = tf.data.Dataset.from_tensor_slices(df_normalized.values).batch(batch_size)

    # Predict water locations using optimized batch inference
    predictions = (model.predict(dataset_full) > 0.5).astype(int)
    df['predicted_water'] = predictions


    # Filter water locations
    water_locations = df[df['predicted_water'] == 1]

    results = water_locations[['X_NAD83UTMz16N', 'Y_NAD83UTMz16N', 'Elev']].rename(columns={'Elev': 'Depth'})
    results['Depth'] = np.abs(results['Depth'])

    print("\nIdentified Water Resources:")
    print(results.head(10).to_string(index=False, justify='left'))

    results.to_csv("identified_water_resources.csv", index=False)
    print("Results saved to identified_water_resources.csv")

except Exception as e:
    print("Error during prediction or result display:", e)
    exit()


Underground Gem Detect Algorithms


In [ ]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Enable Mixed Precision (if using GPU)
tf.keras.mixed_precision.set_global_policy("mixed_float16")


# Step 2: Load and Preprocess Data Efficiently

try:
    # Load datasets
    radiometrics = pd.read_csv('radiometrics.csv')
    magnetics = pd.read_csv('magnetics.csv')
    gravity = pd.read_csv('gravity.csv')
    resistivity = pd.read_csv('resistivity.csv')

    # Merge datasets on common coordinates (X, Y)
    data = pd.merge(radiometrics, magnetics, on=['X', 'Y'])
    data = pd.merge(data, gravity, on=['X', 'Y'])
    data = pd.merge(data, resistivity, on=['X', 'Y'])

    # Feature engineering
    data['K_Th_ratio'] = data['Potassium_NASVD_processed'] / data['Thorium_NASVD_processed']
    data['U_Th_ratio'] = data['Uranium_NASVD_processed'] / data['Thorium_NASVD_processed']


    # Define features and target
    features = ['K_Th_ratio', 'U_Th_ratio', 'Processed_magnetics', 'Gravity_disturbance_Processed', 'Log10Res']
    target = 'Gold_Label'  # Replace with actual labels if available

    # Filter invalid values
    data = data[data['Log10Res'] != -9999]

    # Normalize features
    scaler = MinMaxScaler()
    data_normalized = scaler.fit_transform(data[features])


    # Add target column to normalized data
    data_normalized = pd.DataFrame(data_normalized, columns=features)
    data_normalized[target] = data[target]

except Exception as e:
    print("Error during data loading or preprocessing:", e)
    exit()



In [ ]:
# Step 3: Train ANN Deep Learning Model with Large Data Handling

try:
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        data_normalized[features], data_normalized[target], test_size=0.2, random_state=42
    )

    # Convert to TensorFlow datasets for efficiency
    batch_size = 64
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values)).shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    # Define ANN model
    model = keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(0.3),  # Prevents overfitting
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping callback to prevent overtraining
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

